<a href="https://colab.research.google.com/github/Kallaf/Visual-Odometry/blob/master/VisualOdometry.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mount Drive

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Imports and Global Variables

In [0]:
import os
import sys
import numpy as np
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import cv2
import math
import warnings
import argparse

USE_CUDA = True

LSTM_HIDDEN_SIZE = 550
TIME_STEPS = 1 
K = 100

W0122 20:47:23.564549 140249730111360 deprecation.py:323] From /usr/local/lib/python2.7/dist-packages/tensorflow_core/python/compat/v2_compat.py:68: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
Instructions for updating:
non-resource variables are not supported in the long term


#Deepvonet

## Build Model

In [0]:
class DeepVONet(object):
    def __init__(self,args,data):
        rnn_layers = [tf.nn.rnn_cell.LSTMCell(size) for size in [LSTM_HIDDEN_SIZE, LSTM_HIDDEN_SIZE]]
        multi_rnn_cell = tf.nn.rnn_cell.MultiRNNCell(rnn_layers)

        rnn_inputs = []
        reuse = None
        for stacked_img in data:
            rnn_inputs.append(self.forward(stacked_img, reuse=reuse))
            reuse = True
        rnn_inputs = [tf.reshape(rnn_inputs[i],[-1, 20*6*1024]) for i in range(len(rnn_inputs))]
        assert rnn_inputs[0].shape == (args.bsize, 20*6*1024)

        self.outputs, self.state = tf.nn.static_rnn(cell=multi_rnn_cell,
                                          inputs=rnn_inputs,
                                          dtype=tf.float32)
        assert self.outputs[0].shape == (args.bsize, LSTM_HIDDEN_SIZE)

    def forward(self,x, reuse = None):
            with tf.variable_scope("cnns", reuse=reuse):
                x = tf.layers.conv2d(
                    inputs=x,
                    filters=64,
                    kernel_size=[7, 7],
                    padding="same",
                    strides=2,
                    reuse = reuse,
                    activation=tf.nn.relu, name='conv1')
                x = tf.layers.conv2d(
                        inputs=x,
                        filters=128,
                        kernel_size=[5, 5],
                        padding ="same",
                        strides=2,
                        reuse = reuse,
                        activation=tf.nn.relu, name='conv2')
                x = tf.layers.conv2d(
                        inputs=x,
                        filters=256,
                        kernel_size=[5, 5],
                        padding ="same",
                        strides=2,
                        reuse = reuse,
                        activation=tf.nn.relu, name='conv3')
                x = tf.layers.conv2d(
                        inputs=x,
                        filters=256,
                        kernel_size=[3, 3],
                        padding ="same",
                        strides=1,
                        reuse = reuse,
                        activation=tf.nn.relu, name='conv3_1')
                x = tf.layers.conv2d(
                        inputs=x,
                        filters=512,
                        kernel_size=[3, 3],
                        padding ="same",
                        strides=2,
                        reuse = reuse,
                        activation=tf.nn.relu, name='conv4')
                x = tf.layers.conv2d(
                        inputs=x,
                        filters=512,
                        kernel_size=[3, 3],
                        padding ="same",
                        strides=1,
                        reuse = reuse,
                        activation=tf.nn.relu, name='conv4_1')
                x = tf.layers.conv2d(
                        inputs=x,
                        filters=512,
                        kernel_size=[3, 3],
                        padding ="same",
                        strides=2,
                        reuse = reuse,
                        activation=tf.nn.relu, name='conv5')
                x = tf.layers.conv2d(
                        inputs=x,
                        filters=512,
                        kernel_size=[3, 3],
                        padding ="same",
                        strides=1,
                        reuse = reuse,
                        activation=tf.nn.relu, name='conv5_1')
                x = tf.layers.conv2d(
                        inputs=x,
                        filters=1024,
                        kernel_size=[3, 3],
                        padding ="same",
                        reuse = reuse,
                        strides=2, name='conv6')
            return x


## Train Model

In [0]:
def initialize_uninitialized(sess):
    global_vars = tf.global_variables()
    is_not_initialized = sess.run([tf.is_variable_initialized(var) for var in global_vars])
    not_initialized_vars = [v for (v, f) in zip(global_vars, is_not_initialized) if not f]

    for i in not_initialized_vars: # only for testing
        print(i.name)

    if len(not_initialized_vars):
        sess.run(tf.variables_initializer(not_initialized_vars))

def train_model(data_loader,sess,merged,loss_op,train_op,input_data, labels_,i,test_writer,train_writer):
  print('Current epoch : %d' % data_loader.current_epoch)
  print('step : %d'%i)
  if i % 10 == 0:  # Record summaries and test-set accuracy
      batch_x, batch_y = data_loader.get_next_batch()
      summary, acc = sess.run(
              [merged, loss_op], feed_dict={input_data:batch_x, labels_:batch_y})
      test_writer.add_summary(summary, i)
      print('Accuracy at step %s: %s' % (i, acc))
  else:  # Record train set summaries, and train
      batch_x, batch_y = data_loader.get_next_batch()
      summary, _ = sess.run(
          [merged, train_op], feed_dict={input_data:batch_x, labels_:batch_y})
      train_writer.add_summary(summary, i)
      train_loss = sess.run(loss_op,
              feed_dict={input_data:batch_x, labels_:batch_y})
      print('Train_error at step %s: %s' % (i, train_loss))

def train(args,datapath,epoches,trajectory_length):
  # configuration
  data_loader = VisualOdometryDataLoader(args,datapath,trajectory_length)
  if USE_CUDA:
      sess = tf.Session()
  else:
      config_proto = tf.ConfigProto(device_count = {'GPU': 0})
      sess = tf.Session(config=config_proto)
  pose_size = 6

  # only for gray scale dataset, for colored channels will be 6
  height, width, channels = 384, 1280, 6

  with tf.name_scope('input'):
      # placeholder for input
      input_data = tf.placeholder(tf.float32, [args.bsize, args.time_steps, height, width, channels])
      # placeholder for labels
      labels_ = tf.placeholder(tf.float32, [args.bsize, args.time_steps, pose_size])

  with tf.name_scope('unstacked_input'):
      # Unstacking the input into list of time series
      data = tf.unstack(input_data, args.time_steps, 1)
      # Unstacking the labels into the time series
      pose_labels = tf.unstack(labels_, args.time_steps, 1)


  # Building the RCNN Network which
  # which returns the time series of output layers
  with tf.name_scope('RCNN'):
      model = DeepVONet(args,data)
      (outputs, _)  = (model.outputs, model.state)
  ## Output layer to compute the output
  with tf.name_scope('weights'):
      regression_w = tf.get_variable('regression_w', shape=[LSTM_HIDDEN_SIZE, pose_size], dtype=tf.float32)
  with tf.name_scope('biases'):
      regression_b = tf.get_variable("regression_b", shape=[pose_size], dtype=tf.float32)

  # Pose estimate by multiplication with RCNN_output and Output layer
  with tf.name_scope('Wx_plus_b'):
      pose_estimated = [tf.nn.xw_plus_b(output_state, regression_w, regression_b) for output_state in outputs]
      max_time = len(pose_estimated)

  # Converting the list of tensor into a tensor
  # Probably this is the part that is unnecessary and causing problems (slowing down the computations)

  # Loss function for all the frames in a batch
  with tf.name_scope('loss_l2_norm'):
      position = [pose_es[:,:3] - pose_lab[:,:3] for pose_es, pose_lab in zip(pose_estimated, pose_labels)]
      angles = [pose_es[:,3:6] - pose_lab[:,3:6] for pose_es, pose_lab in zip(pose_estimated, pose_labels)]
      pose_error = (tf.square(position))
      angle_error = (tf.square(angles))
      loss_op = tf.reduce_sum(pose_error + K*angle_error, name='loss')
      tf.summary.scalar('loss_l2_norm', loss_op)

  #optimizer
  with tf.name_scope('train'):
      optimizer = tf.train.AdamOptimizer(learning_rate=args.lr,
            beta1=0.9,
            beta2=0.999,
            epsilon=1e-08,
            use_locking=False,
            name='Adam')
      
      train_op = optimizer.minimize(loss_op)
  # Merge all the summeries and write them out to args.datapath
  # by default ./args.datapath
  merged = tf.summary.merge_all()

  global_vars = tf.global_variables()
  is_not_initialized = sess.run([tf.is_variable_initialized(var) for var in global_vars])
  not_initialized_vars = [v for (v, f) in zip(global_vars, is_not_initialized) if not f]

  if len(not_initialized_vars):
      sess.run(tf.variables_initializer(not_initialized_vars))

  saver = tf.train.Saver()
  initialize_uninitialized(sess)
  train_writer = tf.summary.FileWriter(args.datapath + 'train', sess.graph)
  test_writer = tf.summary.FileWriter(args.datapath + 'test')
  
  i = 0
  while data_loader.current_epoch < epoches:
    train_model(data_loader,sess,merged,loss_op,train_op,input_data, labels_,i,test_writer,train_writer)
    i += 1
  save_path = saver.save(sess, args.datapath + 'model')
  print("Model saved in file: %s" % save_path)
  print("epochs trained: " + str(data_loader.current_epoch))
  train_writer.close()
  test_writer.close()

#Dataset

In [0]:
def default_image_loader(path):
    img = cv2.imread( path )
    if img is not None:
        # Normalizing and Subtracting mean intensity value of the corresponding image
        img = img/np.max(img)
        img = img - np.mean(img)
        img = cv2.resize(img, (1280,384), fx=0, fy=0)
    return img

class VisualOdometryDataLoader(object):
    def __init__(self, args, datapath, trajectory_length,loader=default_image_loader):
        self.args = args
        self._current_initial_frame = 0
        self._current_trajectory_index = 0
        self.current_epoch = 0

        
        self.sequences = [1,4]
        
        self.base_path= datapath 
        self.poses = self.load_poses()
        self.trajectory_length = trajectory_length
        self.loader = loader

    def get_image(self, sequence, index):
        image_path = os.path.join(self.base_path, 'sequences', '%02d' % sequence, 'image_3', '%06d' % index + '.png')
        image = self.loader(image_path)
        return image

    def load_poses(self):
        all_poses = []
        for sequence in self.sequences:
            with open(os.path.join(self.base_path, 'poses/',  ('%02d' % sequence) + '.txt')) as f:
                poses = np.array([[float(x) for x in line.split()] for line in f], dtype=np.float32)
                all_poses.append(poses)
        return all_poses

    def _set_next_trajectory(self):
        if (self._current_trajectory_index < self.trajectory_length-1):
            self._current_trajectory_index += 1
        else:
            self.current_epoch += 1
            self._current_trajectory_index = 0

        self._current_initial_frame = 0

    def get_next_batch(self):
        img_batch = []
        label_batch = []

        poses = self.poses[self._current_trajectory_index]

        for j in range(self.args.bsize):
            img_stacked_series = []
            labels_series = []

            read_img = self.get_image(self.sequences[self._current_trajectory_index], self._current_initial_frame + self.args.time_steps)
            if (read_img is None):self._set_next_trajectory()

            for i in range(self._current_initial_frame, self._current_initial_frame + self.args.time_steps):
                img1 = self.get_image(self.sequences[self._current_trajectory_index], i)
                img2 = self.get_image(self.sequences[self._current_trajectory_index], i+1)
                img_aug = np.concatenate([img1, img2], -1)
                img_stacked_series.append(img_aug)
                pose = self.get6DoFPose(poses[i,:]) - self.get6DoFPose(poses[self._current_initial_frame,:])
                labels_series.append(pose)
            img_batch.append(img_stacked_series)
            label_batch.append(labels_series)
            self._current_initial_frame += self.args.time_steps
        label_batch = np.array(label_batch)
        img_batch = np.array(img_batch)
        return img_batch, label_batch

    def isRotationMatrix(self,R):
        Rt = np.transpose(R)
        shouldBeIdentity = np.dot(Rt, R)
        I = np.identity(3, dtype = R.dtype)
        n = np.linalg.norm(I - shouldBeIdentity)
        return n < 1e-6

    def rotationMatrixToEulerAngles(self,R):
        assert(self.isRotationMatrix(R))
        sy = math.sqrt(R[0,0] * R[0,0] +  R[1,0] * R[1,0])
        singular = sy < 1e-6

        if  not singular :
            x = math.atan2(R[2,1] , R[2,2])
            y = math.atan2(-R[2,0], sy)
            z = math.atan2(R[1,0], R[0,0])
        else :
            x = math.atan2(-R[1,2], R[1,1])
            y = math.atan2(-R[2,0], sy)
            z = 0

        return np.array([x, y, z])

    def get6DoFPose(self, p):
        pos = np.array([p[3], p[7], p[11]])
        R = np.array([[p[0], p[1], p[2]], [p[4], p[5], p[6]], [p[8], p[9], p[10]]])
        angles = self.rotationMatrixToEulerAngles(R)
        return np.concatenate((pos, angles))


# Main

## Args Class

In [0]:
class MyArgs():
    def __init__(self,datapath,bsize,trajectory_length,lr=0.001, time_steps=100,train_iter = 5):
        self.datapath = datapath
        self.bsize = bsize
        self.trajectory_length = trajectory_length
        self.lr = lr
        self.time_steps = time_steps
        self.train_iter = train_iter

## main code

In [0]:
args = MyArgs(datapath='/content/drive/My Drive/dataset/',
              bsize=1,
              trajectory_length=2,
              lr=0.0001,
              train_iter = 5,
              time_steps=TIME_STEPS)



train(args,args.datapath,args.train_iter,args.trajectory_length)

W0122 20:47:24.134143 140249730111360 deprecation.py:323] From <ipython-input-3-dd4023bd84c1>:3: __init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
W0122 20:47:24.140674 140249730111360 deprecation.py:323] From <ipython-input-3-dd4023bd84c1>:4: __init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
W0122 20:47:24.144129 140249730111360 deprecation.py:323] From <ipython-input-3-dd4023bd84c1>:28: conv2d (from tensorflow.python.layers.convolutional) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
W0122 20:47:24.146157 140249730111360 deprecation.py:323

Current epoch : 0
step : 0
Accuracy at step 0: 27.314495
Current epoch : 0
step : 1
Train_error at step 1: 24.480341
Current epoch : 0
step : 2
Train_error at step 2: 12.221834
Current epoch : 0
step : 3
Train_error at step 3: 10.031147
Current epoch : 0
step : 4
Train_error at step 4: 1.411536
Current epoch : 0
step : 5
Train_error at step 5: 5.5894294
Current epoch : 0
step : 6
Train_error at step 6: 5.6599216
Current epoch : 0
step : 7
Train_error at step 7: 3.0165963
Current epoch : 0
step : 8
Train_error at step 8: 0.6806759
Current epoch : 0
step : 9
Train_error at step 9: 1.5254214
Current epoch : 0
step : 10
Accuracy at step 10: 1.5194417
Current epoch : 0
step : 11
Train_error at step 11: 2.1226888
Current epoch : 0
step : 12
Train_error at step 12: 1.023267
Current epoch : 0
step : 13
Train_error at step 13: 0.49024555
Current epoch : 0
step : 14
Train_error at step 14: 0.75737786
Current epoch : 0
step : 15
Train_error at step 15: 1.1462274
Current epoch : 0
step : 16
Train_

# Test

In [0]:
args = MyArgs(datapath='/content/drive/My Drive/dataset/',
              bsize=1,
              trajectory_length=2,
              train_iter = 1,
              time_steps=TIME_STEPS)

# configuration
config_proto = tf.ConfigProto(device_count = {'GPU': 0})
sess = tf.Session(config=config_proto)
data_loader = VisualOdometryDataLoader(args,args.datapath,args.trajectory_length)
""" input_batch must be in shape of [?, TIME_STEPS, 384, 1280, 6] """
#tf.reset_default_graph()
print('Restoring Entire Session from checkpoint : %s'%args.datapath+"model.meta")
imported_meta = tf.train.import_meta_graph(args.datapath + "model.meta")
print('Success')
imported_meta.restore(sess, tf.train.latest_checkpoint(args.datapath))
input_data = tf.get_default_graph().get_tensor_by_name("input/Placeholder:0")
# placeholder for labels
labels_ = tf.get_default_graph().get_tensor_by_name("input/Placeholder_1:0")
loss_op = tf.get_default_graph().get_tensor_by_name("loss_l2_norm/loss:0")
poses = []
poses.append(tf.get_default_graph().get_tensor_by_name("Wx_plus_b/xw_plus_b:0"))
# poses.append(tf.get_default_graph().get_tensor_by_name("Wx_plus_b/xw_plus_b_1:0"))
# poses.append(tf.get_default_graph().get_tensor_by_name("Wx_plus_b/xw_plus_b_2:0"))
# poses.append(tf.get_default_graph().get_tensor_by_name("Wx_plus_b/xw_plus_b_3:0"))
# poses.append(tf.get_default_graph().get_tensor_by_name("Wx_plus_b/xw_plus_b_4:0"))
while data_loader.current_epoch < args.train_iter:
    input_, ground_truth_batch = data_loader.get_next_batch()
    output = sess.run(poses, feed_dict={input_data:input_})
    print('Current epoch : %d' % data_loader.current_epoch)
    print('output length : %d' % len(output))
    for i in range(len(output)):
        fh = open("output_file","a")
        fh.write("%f %f %f %f %f %f\n"%(ground_truth_batch[0,i,0],
                                        ground_truth_batch[0,i,1],
                                        ground_truth_batch[0,i,2],
                                        ground_truth_batch[0,i,3],
                                        ground_truth_batch[0,i,4],
                                        ground_truth_batch[0,i,5]))
        fh.close()
        fh = open("estimated","a")
        fh.write("%f %f %f %f %f %f\n"%(output[i][0,0],
                                        output[i][0,1],
                                        output[i][0,2],
                                        output[i][0,3],
                                        output[i][0,4],
                                        output[i][0,5]))
        fh.close()

Restoring Entire Session from checkpoint : /content/drive/My Drive/dataset/model.meta
Success
Current epoch : 0
output length : 1
Current epoch : 0
output length : 1
Current epoch : 0
output length : 1
Current epoch : 0
output length : 1
Current epoch : 0
output length : 1
Current epoch : 0
output length : 1
Current epoch : 0
output length : 1
Current epoch : 0
output length : 1
Current epoch : 0
output length : 1
Current epoch : 0
output length : 1
Current epoch : 0
output length : 1
Current epoch : 0
output length : 1
Current epoch : 0
output length : 1
Current epoch : 0
output length : 1
Current epoch : 0
output length : 1
Current epoch : 0
output length : 1
Current epoch : 0
output length : 1
Current epoch : 0
output length : 1
Current epoch : 0
output length : 1
Current epoch : 0
output length : 1
Current epoch : 0
output length : 1
Current epoch : 0
output length : 1
Current epoch : 0
output length : 1
Current epoch : 0
output length : 1
Current epoch : 0
output length : 1
Curren